In [515]:
import numpy as np
import pandas as pd

from tqdm import tqdm

In [332]:
diner_cols = [
        #'idx',
        'diner_name',           # 가게이름 
        'diner_category',       # 가게 카테고리
        'diner_menu',           # 가게 메뉴
        'diner_review_cnt',     # 가게의 평점 개수 
        'diner_review_avg',     # 가게의 평점 평균
        'diner_review_tags',    # 리뷰 태그
        'diner_address',        # 가게 주소    
        'diner_lon',            # 가게 위도
        'diner_lat',            # 가게 경도
        'diner_url',            # 가게 URL
        'diner_open_time'       # 가게 오픈시간
       ]

review_cols = [
        #'idx',
        'reviewer_avg',         # 리뷰어의 평점 평균
        'reviewer_review_cnt',  # 리뷰어의 리뷰 개수
        'reviewer_review_score',# 리뷰어가 남긴 평점
        'reviewer_review_date', # 리뷰를 남긴 날짜
        'diner_idx'             # 가게 인덱스
        ]

In [333]:
# df 생성
diner_df = pd.DataFrame(columns = diner_cols)
review_df = pd.DataFrame(columns = review_cols)

In [668]:
# read_csv
df = pd.read_csv("./KakaoRok_Dev/OldKakaoRok/matki_DB.csv")

C:\Users\isl12\AppData\Local\Temp\ipykernel_3156\984553786.py:1: DtypeWarning: Columns (14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./KakaoRok_Dev/OldKakaoRok/matki_DB.csv")


In [669]:
# 위도, 경도의 데이터 처리
df.loc[df["lon"] == "None", "lon"] = 0.0
df.loc[df["lat"] == "None", "lat"] = 0.0
df["lon"] = df["lon"].astype(float)
df["lat"] = df["lat"].astype(float)

# 위도상으로 120 미만에 있는 모든 것들이 위도,경도가 바뀜
swap_series = df.loc[df.query("lon < 120").index, "lon"]
df.loc[df.query("lon < 120").index, "lon"] = df.loc[df.query("lon < 120").index, "lat"]
df.loc[df.query("lon < 120").index, "lat"] = swap_series

In [681]:
# 가게이름에 대한 키가 없기 때문에 해당 파일에서 구분할 수 있는 기준 정의 
usecols = ['name', 'addresse',  'review_num', 'score_min', 'cat1', 'likePoint', 'likePointCnt']
li = df[usecols].value_counts().index
diner_cnt = 0 

for idx in tqdm(li) :
    # diner_df에 들어갈 값 저장
    diner_name, diner_address, diner_review_cnt, diner_review_avg, diner_category = idx[:5]
    diner_review_tags = "@".join(["".join(i) for i in zip([j for j in idx[5].split("@")], [k for k in idx[6].split("@")])])
    
    # 그의 위도,경도,URL, 오픈 시간
    ori_df = df.query(f"name == '{diner_name}' and addresse == '{diner_address}'")
    
    # loc를 통해 데이터 삽입
    diner_df.loc[diner_cnt] = [
                            diner_name,
                            diner_category,
                            ori_df.iloc[-1]["cat2"],
                            diner_review_cnt,
                            diner_review_avg,
                            diner_review_tags,
                            diner_address,
                            ori_df.iloc[-1]["lon"],
                            ori_df.iloc[-1]["lat"],
                            ori_df.iloc[-1]["url"],
                            ori_df.iloc[-1]["open_time"]
                            ]
    diner_cnt += 1

100%|█████████████████████████████████████████████████████████████████████████████| 6415/6415 [00:47<00:00, 134.05it/s]


In [728]:
review_cnt = 0

# 가게이름과 주소를 통한 최소한의 중복 제거 후 반복 실행
for i in tqdm(diner_df.drop_duplicates(subset = ["diner_name", "diner_address"], keep = "last").index) :
    # query를 통해 가게 이름과 주소로 서치
    ori_df = df.query(f"name == '{diner_df.loc[i, 'diner_name']}' and addresse == '{diner_df.loc[i, 'diner_address']}'")
    
    # 리뷰 평점 평균, 리뷰자가 올린 평점, 리뷰올린 날짜로 중복 제거
    for j in ori_df.drop_duplicates(subset = ["rate", "reviewAt","reviews_date"], keep = "last").index :
        # 데이터 삽입
        review_df.loc[review_cnt] = [
                                ori_df.loc[j, "rate"],
                                ori_df.loc[j, "reviewerCnt"],
                                ori_df.loc[j, "reviewAt"],
                                ori_df.loc[j, "reviews_date"],
                                i
                                ]
        review_cnt += 1

100%|██████████████████████████████████████████████████████████████████████████████| 5206/5206 [03:12<00:00, 27.01it/s]


In [729]:
# 두 데이터 프레임을 시트로 구분하여 엑셀로 저장
writer = pd.ExcelWriter('matki_DB.xlsx', engine='xlsxwriter')
diner_df.to_excel(writer, sheet_name= 'diner')
review_df.to_excel(writer, sheet_name= 'review')
writer.save()